# Transfer Service

[Storage Transfer Service](https://cloud.google.com/storage-transfer-service)^[I SSB kaller vi tjenesten for *Transfer Service*, men du kan oppleve at Google kaller den litt forskjellige ting. Den blir omtalt som *Storage Transfer Service* noen steder, mens i *Google Cloud Console* blir den omtalt som *Data Transfer* eller *Storage Transfer*] er en Google-tjeneste for å flytte data mellom lagringsområder. I SSB bruker vi hovedsakelig tjenesten til å: 

1. Flytte data mellom et Dapla-team sine bøtter på Dapla og deres lagringsområde i prodsonen. 

2. Flytte data mellom bøtter på Dapla^[Flytting av data mellom bøtter krever at prosjektets Transfer Service har tilgang til begge bøttene.].

Tjenesten støtter både automatiserte og ad-hoc overføringer, og den inkluderer et brukergrensesnitt for å sette opp og administrere overføringene i [Google Cloud Console (GCC)](https://console.cloud.google.com/).

Under viser vi hvordan du kan sette dette opp selv i dine prosjekter.

## To Transfer Services

Hvert Dapla-team som har bedt om å få *skrudd på* Transfer Service vil få en Transfer Service for hvert av de to prosjektene sine. Det skyldes at det kun er teamets **data-admins** som skal ha tilgang til å sette opp overføringer knyttet til kildedata, mens alle i teamet skal ha tilgang til å sette opp overføringer fra produktprosjektet.

## Prodsonen

Overføring av data mellom prodsonen og Dapla skjer via dedikerte lagringsområder i prodsonen. Når et team opprettes 

```{.yaml filename="/ssb/cloud_sync/dapla-example/"}
ssb
│
├── cloud_sync
│   │
│   └── dapla-example
│       │
│       ├── kilde
│       │   │
│       │   │── tilsky
│       │   │
│       │   └── frasky
│       │
│       └── produkt
│           │
│           │── tilsky
│           │
│           └── frasky

```

In [1]:
# | echo: false

# Lager et diagram
from diagrams import Cluster, Diagram, Edge
from diagrams.gcp.storage import GCS
from diagrams.generic.storage import Storage
from diagrams.generic.blank import Blank

graph_attr_cluster = {
    "fontsize": "8",
    "bgcolor": "lightyellow",
    "gradientangle": "360"
    # "style": 'filled',
    # "label": "test",
    # "fontcolor": "blue",
}

graph_attr_diagram = {
    # "fontsize": "45",
    "bgcolor": "transparent"
}

with Diagram(
    "", show=False, filename="ts-kilde", graph_attr={}, direction="TB"
):
    with Cluster("DAPLA", graph_attr={"fontsize": "20"}):
        Blank("")

        with Cluster("kilde", graph_attr=graph_attr_cluster):
            dkf = GCS("synk_ned")
            dkt = GCS("synk_opp")

        with Cluster("produkt", graph_attr=graph_attr_cluster):
            dpf = GCS("synk_ned")
            dpt = GCS("synk_opp")

    with Cluster("PRODSONE", graph_attr={"fontsize": "20", "bgcolor": "lightgrey"}):
        Blank("/ssb/cloud_sync/")

        with Cluster("kilde", graph_attr=graph_attr_cluster):
            okf = Storage("frasky")
            okt = Storage("tilsky")

        with Cluster("produkt", graph_attr=graph_attr_cluster):
            opf = Storage("frasky")
            opt = Storage("tilsky")

        edge_admins = Edge(xlabel="data-admins")
        edge_devs = Edge(xlabel="developers")

    dkf >> edge_admins >> okf
    okt >> dkt
    dpf >> edge_devs >> opf
    opt >> dpt


In [2]:
#| echo: false

# Cropper bildet så det skal ta mindre plass
from PIL import Image

# Open the image
img = Image.open("ts-kilde.png")

# Define the coordinates for the cropping box
# The box is defined as (left, upper, right, lower)
left = 200
upper = 200
right = 1150
lower = 800
cropping_box = (left, upper, right, lower)

# Crop the image
cropped_img = img.crop(cropping_box)

# Save the cropped image
cropped_img.save("ts-kilde-cropped.png")

![blabla](ts-kilde-cropped.png){fig-alt="Bilde av SSB sitt intranett der Dapla-lenken er uthevet" #fig-ts2}

jdhfdsjkfhkjdshfkjdsh